## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1369. Get the Second Most Recent Activity (Hard)**

**Table: UserActivity**

| Column Name   | Type    |
|---------------|---------|
| username      | varchar |
| activity      | varchar |
| startDate     | Date    |
| endDate       | Date    |

This table may contain duplicates rows.
This table contains information about the activity performed by each user in a period of time.
A person with username performed an activity from startDate to endDate.
 
**Write a solution to show the second most recent activity of each user.**

If the user only has one activity, return that one. A user cannot perform more than one activity at the same time.

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**UserActivity table:**

| username   | activity     | startDate   | endDate     |
|------------|--------------|-------------|-------------|
| Alice      | Travel       | 2020-02-12  | 2020-02-20  |
| Alice      | Dancing      | 2020-02-21  | 2020-02-23  |
| Alice      | Travel       | 2020-02-24  | 2020-02-28  |
| Bob        | Travel       | 2020-02-11  | 2020-02-18  |

**Output:** 
| username   | activity     | startDate   | endDate     |
|------------|--------------|-------------|-------------|
| Alice      | Dancing      | 2020-02-21  | 2020-02-23  |
| Bob        | Travel       | 2020-02-11  | 2020-02-18  |

**Explanation:** 
The most recent activity of Alice is Travel from 2020-02-24 to 2020-02-28, before that she was dancing from 2020-02-21 to 2020-02-23.
Bob only has one record, we just take that one.

In [0]:
user_activity_data_1369 = [
    ("Alice", "Travel", "2020-02-12", "2020-02-20"),
    ("Alice", "Dancing", "2020-02-21", "2020-02-23"),
    ("Alice", "Travel", "2020-02-24", "2020-02-28"),
    ("Bob", "Travel", "2020-02-11", "2020-02-18")
]

user_activity_columns_1369 = ["username", "activity", "startDate", "endDate"]
user_activity_df_1369 = spark.createDataFrame(user_activity_data_1369, user_activity_columns_1369)
user_activity_df_1369.show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice|  Travel|2020-02-12|2020-02-20|
|   Alice| Dancing|2020-02-21|2020-02-23|
|   Alice|  Travel|2020-02-24|2020-02-28|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+



In [0]:
user_activity_df_1369 = user_activity_df_1369\
                                .withColumn("startDate", to_date(col("startDate"), "yyyy-MM-dd")) \
                                .withColumn("endDate", to_date(col("endDate"), "yyyy-MM-dd"))

In [0]:
rank_window = Window.partitionBy("username").orderBy(col("startDate").desc())
count_window = Window.partitionBy("username")

In [0]:
df_ranked_1369 = user_activity_df_1369\
                .withColumn("row_num", row_number().over(rank_window)) \
                .withColumn("activity_count", count("*").over(count_window))

In [0]:
df_ranked_1369\
    .filter(
    (col("row_num") == 2) | ((col("row_num") == 1) & (col("activity_count") == 1)))\
        .select("username", "activity", "startDate", "endDate").show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice| Dancing|2020-02-21|2020-02-23|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+

